# TFX for OG problem

An example building a ML pipeline using Tensorflow TFX

In [66]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.1
TFX version: 1.7.1


In [67]:
import os

PIPELINE_NAME = "og-model"

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = "og-model-tfdv-schema"
PIPELINE_NAME = "og-model"

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.


### Prepare example data

We will download the example dataset for use in our TFX pipeline. 

Because the TFX ExampleGen component reads inputs from a directory, we need to create a directory and copy the dataset to it.

In [6]:
import urllib.request
import tempfile

DATA_ROOT = '/home/onwunalu/data/datasets/machine-learning/og-data'
_data_filepath = os.path.join(DATA_ROOT, "og_data_40_40_1.csv")

Take a quick look at what the raw data looks like.

In [7]:
!head {_data_filepath}

X,Y,F
0,0,8.342349
1,0,9.585238
2,0,12.00188
3,0,11.96141
4,0,11.42057
5,0,11.17394
6,0,12.27702
7,0,11.30125
8,0,12.72968


You should be able to see 2 features 'X' and 'Y'. F is the target value that we will try to predict

### Generate a preliminary schema

TFX pipelines are defined using Python APIs. We will create a pipeline to generate a schema from the input examples automatically. This schema can be reviewed by a human and adjusted as needed. Once the schema is finalized it can be used for training and example validation in later tasks.

In addition to CsvExampleGen which is used in Simple TFX Pipeline Tutorial, we will use StatisticsGen and SchemaGen:

StatisticsGen calculates statistics for the dataset.
SchemaGen examines the statistics and creates an initial data schema. See the guides for each component or TFX components tutorial to learn more on these components.
 

### Write a pipeline definition

We define a function to create a TFX pipeline. A Pipeline object represents a TFX pipeline which can be run using one of pipeline orchestration systems that TFX supports.

In [8]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Run the pipeline

We will use LocalDagRunner as in the previous tutorial.

In [9]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "SchemaGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.schema_gen.executor.Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metada

INFO:absl:Processing input csv data /home/onwunalu/data/datasets/machine-learning/og-data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 4 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/og-model-tfdv-schema/CsvExampleGen/examples/4"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:23748,xor_checksum:1654054099,sum_checksum:1654054099"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "og-model-tfdv-schema:2022-06-04T00:04:33.339121:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key

INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines/og-model-tfdv-schema/StatisticsGen/statistics/5/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines/og-model-tfdv-schema/StatisticsGen/statistics/5/Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 5 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'statistics': [Artifact(artifact: uri: "pipelines/og-model-tfdv-schema/StatisticsGen/statistics/5"
custom_properties {
  key: "name"
  value {
    string_value: "og-model-tfdv-schema:2022-06-04T00:04:33.339121:StatisticsGen:statistics:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.7.1"
  }
}
, artifact_type: name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_typ

You should see "INFO:absl:Component SchemaGen is finished." if the pipeline finished successfully.

We will examine the output of the pipeline to understand our dataset.


### Review outputs of the pipeline

As explained in the previous tutorial, a TFX pipeline produces two kinds of outputs, artifacts and a metadata DB(MLMD) which contains metadata of artifacts and pipeline executions. We defined the location of these outputs in the above cells. By default, artifacts are stored under the pipelines directory and metadata is stored as a sqlite database under the metadata directory.

You can use MLMD APIs to locate these outputs programatically. First, we will define some utility functions to search for the output artifacts that were just produced.

In [10]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_artifacts_dict(metadata, latest_execution.id,
                                          [metadata_store_pb2.Event.OUTPUT])

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

Now we can examine the outputs from the pipeline execution.

In [11]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

INFO:absl:MetadataStore with DB connection initialized


In [34]:
SCHEMA_PIPELINE_NAME

'og-model-tfdv-schema'

It is time to examine the outputs from each component. As described above, Tensorflow Data Validation(TFDV) is used in StatisticsGen and SchemaGen, and TFDV also provides visualization of the outputs from these components.

In this tutorial, we will use the visualization helper methods in TFX which use TFDV internally to show the visualization.

Examine the output from StatisticsGen

In [12]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

You can see various stats for the input data. These statistics are supplied to SchemaGen to construct an initial schema of data automatically.

### Examine the output from SchemaGen

In [13]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'F',FLOAT,required,,-
'X',INT,required,,-
'Y',INT,required,,-


This schema is automatically inferred from the output of StatisticsGen. You should be able to see 1 FLOAT feature and 2 INT features.

### Export the schema for future use

In [14]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

'schema/schema.pbtxt'

In [19]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

Schema at schema-----
feature {
  name: "F"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "X"
  value_count {
    min: 1
    max: 40
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Y"
  value_count {
    min: 1
    max: 40
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}


### Update schema and add min and max values for X and Y variables
Note that we have edited the schema to add the min and max of the X and Y variables.

In [22]:
# NEW: Import the schema.
schema_path=SCHEMA_PATH
schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

### Create a pipeline

In [73]:
_module_file = 'og_utils.py'

In [96]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

# Specify features that we will use.
_FEATURE_KEYS = [
    'X', 'Y',
]
_LABEL_KEY = 'F'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  outputs[_LABEL_KEY] = inputs[_LABEL_KEY]
  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 3 possible values, we use a hard-coded
  # table for simplicity.
  #table_keys = ['Adelie', 'Chinstrap', 'Gentoo']
  #initializer = tf.lookup.KeyValueTensorInitializer(
  #    keys=table_keys,
  ##    values=tf.cast(tf.range(len(table_keys)), tf.int64),
  #    key_dtype=tf.string,
  #    value_dtype=tf.int64)
  #table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  #outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  #outputs = keras.layers.Dense(3)(d) 
  outputs = keras.layers.Dense(1)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss='mse', #tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.MeanSquaredError()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting og_utils.py


### Write a pipeline definition

In [97]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Run the pipeline

In [98]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/onwunalu/codelib/python/tfx-tutorials/og-pipeline/og_utils.py' (including modules: ['penguin_utils', 'og_utils']).
INFO:absl:User module package has hash fingerprint version 633c8047e467ea832cd4cd035b5ae4ebee704ec88dc6b7eb375a68a56ade56f7.
INFO:absl:Executing: ['/home/onwunalu/.pyenv/versions/tfx/bin/python', '/tmp/tmpt0fucce8/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp8ksr_7lc', '--dist-dir', '/tmp/tmpaf38grf9']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/og-model/_wheels/tfx_user_code_Transform-0.0+633c8047e467ea832cd4cd035b5ae4ebee704ec88dc6b7eb375a68a56ade56f7-py3-none-any.whl'; target user module is 'og_utils'.
INFO:absl:Full user module path is 'og_utils@pipelines/og-model/_wheels/tfx_user_code_Transform-0.0+633c8047e467ea832cd4cd035b5ae4ebee704ec8

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying penguin_utils.py -> build/lib
copying og_utils.py -> build/lib
installing to /tmp/tmp8ksr_7lc
running install
running install_lib
copying build/lib/penguin_utils.py -> /tmp/tmp8ksr_7lc
copying build/lib/og_utils.py -> /tmp/tmp8ksr_7lc
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp8ksr_7lc/tfx_user_code_Transform-0.0+633c8047e467ea832cd4cd035b5ae4ebee704ec88dc6b7eb375a68a56ade56f7-py3.8.egg-info
runnin

INFO:absl:Successfully built user code wheel distribution at 'pipelines/og-model/_wheels/tfx_user_code_Trainer-0.0+633c8047e467ea832cd4cd035b5ae4ebee704ec88dc6b7eb375a68a56ade56f7-py3-none-any.whl'; target user module is 'og_utils'.
INFO:absl:Full user module path is 'og_utils@pipelines/og-model/_wheels/tfx_user_code_Trainer-0.0+633c8047e467ea832cd4cd035b5ae4ebee704ec88dc6b7eb375a68a56ade56f7-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "ExampleValidator"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_validator.executor.Executor"
    }
  }
}
executor_specs {
  key: "Pusher"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.pusher.executor.Executor"
    }
  }
}
executor_specs {
  k

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying penguin_utils.py -> build/lib
copying og_utils.py -> build/lib
installing to /tmp/tmpz930vya0
running install
running install_lib
copying build/lib/penguin_utils.py -> /tmp/tmpz930vya0
copying build/lib/og_utils.py -> /tmp/tmpz930vya0
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpz930vya0/tfx_user_code_Trainer-0.0+633c8047e467ea832cd4cd035b5ae4ebee704ec88dc6b7eb375a68a56ade56f7-py3.8.egg-info
running install_scripts


INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Going to run a new execution 41
INFO:absl:Going to run a new execution: ExecutionInfo(execution_id=41, input_dict={}, output_dict=defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/og-model/CsvExampleGen/examples/41"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:23748,xor_checksum:1654054099,sum_checksum:1654054099"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "og-model:2022-06-05T21:46:34.332304:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DATASET
)]}), exec_properties={'output

...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature F has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature X has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Y has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Model: "model_3"
INFO:absl:__________________________________________________________________________________________________
INFO:absl: Layer (type)                   Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl: X (InputLayer)                 [(None, 1)]          0           []                               
INFO:absl:                                                                                                  
INFO:absl: Y (InputLayer)                 [(None, 1)]          0           []                               
INFO:absl:                               

100/100 [==============================] - 0s 2ms/step - loss: 4652.9043 - mean_squared_error: 4652.9043 - val_loss: 1534.2926 - val_mean_squared_error: 1534.2926
INFO:tensorflow:Assets written to: pipelines/og-model/Trainer/model/46/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/og-model/Trainer/model/46/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/og-model/Trainer/model/46/Format-Serving. ModelRun written to pipelines/og-model/Trainer/model_run/46
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 46 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/og-model/Trainer/model/46"
custom_properties {
  key: "name"
  value {
    string_value: "og-model:2022-06-05T21:46:34.332304:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.7.1"
  }
}
, artifact_type: name: "Model"
base_type: MODEL
)], 'model_run': [Artifact(artifact: uri: "pipelines/og-model/Trainer/model_run/46"
custom_properties {
  key: "name"
  value {
    string_value: "og-model:2022-06-05T21:46:34.332304:Trainer:model_run:0"
  }
}
custom_properties 

In [99]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

serving_model/og-model
serving_model/og-model/1654482986
serving_model/og-model/1654482986/keras_metadata.pb
serving_model/og-model/1654482986/variables
serving_model/og-model/1654482986/variables/variables.data-00000-of-00001
serving_model/og-model/1654482986/variables/variables.index
serving_model/og-model/1654482986/assets
serving_model/og-model/1654482986/saved_model.pb
serving_model/og-model/1654483045
serving_model/og-model/1654483045/keras_metadata.pb
serving_model/og-model/1654483045/variables
serving_model/og-model/1654483045/variables/variables.data-00000-of-00001
serving_model/og-model/1654483045/variables/variables.index
serving_model/og-model/1654483045/assets
serving_model/og-model/1654483045/saved_model.pb
serving_model/og-model/1654483622
serving_model/og-model/1654483622/keras_metadata.pb
serving_model/og-model/1654483622/variables
serving_model/og-model/1654483622/variables/variables.data-00000-of-00001
serving_model/og-model/1654483622/variables/variables.index
servi

In [100]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default



The given SavedModel SignatureDef contains the following input(s):
  inputs['examples'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_examples:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


We can load the exported model and try some inferences with a few examples.




In [101]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path
print(model_path)
loaded_model = tf.keras.models.load_model(model_path, compile=False)
inference_fn = loaded_model.signatures['serving_default']

serving_model/og-model/1654483622


In [114]:
# Prepare an example and run inference.
features = {
  'X': tf.train.Feature(int64_list=tf.train.Int64List(value=[20])),
  'Y': tf.train.Feature(int64_list=tf.train.Int64List(value=[20])),
}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

[[47.508644]]
